In [1]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
import requests
from bs4 import BeautifulSoup

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
url = 'https://delwatergap.net/Lyrics-1'
page = requests.get(url).content

In [4]:
bs = BeautifulSoup(page)

In [34]:
a = bs.find('div',attrs={'class':'page_content clearfix'}).find_all('a')[1:]
hrefs = [x['href'] for x in a]

In [141]:
hrefs.remove('High-Tops-Stripped')

In [142]:
song_urls = ['https://delwatergap.net/'+href for href in hrefs]

In [143]:
song_urls

['https://delwatergap.net/Be-My-Own-Bonus-Track',
 'https://delwatergap.net/Chastain',
 'https://delwatergap.net/Cut-The-Rope',
 'https://delwatergap.net/Deirdre-Pt-I',
 'https://delwatergap.net/Don-t-Let-Me',
 'https://delwatergap.net/Don-t-Read-The-Mirror',
 'https://delwatergap.net/Don-t-Say-Nothing',
 'https://delwatergap.net/High-Tops',
 'https://delwatergap.net/Homeless',
 'https://delwatergap.net/I-Am-Drunk-And-She-Is-Insane',
 'https://delwatergap.net/In-The-Yard',
 'https://delwatergap.net/Laid-Down-My-Arms',
 'https://delwatergap.net/Lamplight',
 'https://delwatergap.net/Let-s-Pretend',
 'https://delwatergap.net/Lost-My-Cat-Put-in-a-Cage',
 'https://delwatergap.net/Love-Song-For-Lady-Earth-Sasha',
 'https://delwatergap.net/Rockman-s-Pier',
 'https://delwatergap.net/Theory-of-Emotion',
 'https://delwatergap.net/To-Philly',
 'https://delwatergap.net/Still-in-Love',
 'https://delwatergap.net/Vanessa']

In [144]:
lyrics = []
for song in song_urls:
    lyrics.append([x.get_text().replace('\n',' ').replace('\u2028', ' ') for x in BeautifulSoup(requests.get(song).content).find_all('h2')])

In [145]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = [x[:-2] for x in list(sent_to_words(lyrics))]

print(data_words[0])

['we', 'never', 'sleep', 'at', 'night', 'but', 'we', 'll', 'spend', 'the', 'days', 'in', 'bed', 'and', 'you', 'walk', 'down', 'the', 'stairs', 'ever', 'so', 'gingerly', 'could', 've', 'lived', 'but', 'chose', 'to', 'love', 'you', 'instead', 'fix', 'me', 'your', 'poison', 'gin', 'for', 'your', 'molly', 'lime', 'for', 'your', 'bruises', 'and', 'chalk', 'for', 'your', 'holly', 'tell', 'me', 'you', 're', 'sleeping', 'can', 'if', 'you', 'won', 'yeah', 'you', 'could', 'be', 'my', 'own', 'the', 'way', 'that', 'you', 'smile', 'when', 'something', 'pleasant', 'is', 'up', 'in', 'your', 'mind', 'and', 'you', 'put', 'your', 'tiny', 'lips', 'to', 'my', 'ear', 'we', 'could', 'break', 'this', 'old', 'routine', 'this', 'apartment', 'you', 'and', 'me', 'we', 'could', 'leave', 'this', 'town', 'my', 'dear', 'rats', 'in', 'your', 'armchair', 'burrs', 'in', 'your', 'cotton', 'baby', 'turn', 'my', 'engines', 'from', 'polished', 'to', 'rotten', 'promise', 'you', 'll', 'keep', 'me', 'in', 'box', 'by', 'the', 

In [146]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['never sleep night will spend day bed walk stair ever so gingerly could ve live choose love instead fix poison gin molly lime bruise chalk holly tell re sleep can win could own way smile when pleasant up mind put tiny lip ear could break old routine apartment could leave town dear rat armchair burr cotton baby turn engine polished rotten promise will keep box phone could own would burn crow field sing sleep win ever walk cause car just blockand only week away travel sea love where move heart where go clothe floor bruise show airplane can take will still own bell boathouse champagne ice just movie parent cry family gather re get stone could own left picture box road could own', 'just day will look get mistake man job take session lunch break win believe apartment will cost live right freeway watch tv dollar store sock get much day convince been talk too much picture hour replay pretty lonely just write song when know friend move away just see picture mom when age as hard think happy da

In [147]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [148]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,           # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [149]:
# Define Search Param
search_params = {'n_components': [2, 3, 4, 5, 6], 
                 'learning_decay': [.8, .9, .95]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                   

In [150]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 2}
Best Log Likelihood Score:  -191.85278100280397
Model Perplexity:  6.97157320366409


In [151]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(lyrics))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,dominant_topic
Doc0,0.89,0.11,0
Doc1,0.95,0.05,0
Doc2,0.9,0.1,0
Doc3,0.75,0.25,0
Doc4,0.86,0.14,0
Doc5,0.07,0.93,1
Doc6,0.63,0.37,0
Doc7,0.86,0.14,0
Doc8,0.92,0.08,0
Doc9,0.31,0.69,1


In [152]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,0,17
1,1,4


In [153]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel


//anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x           y  topics  cluster       Freq
topic                                                   
0     -60.312317  141.369629       1        1  53.657195
1      60.312561 -141.369141       2        1  46.342805, topic_info=  Category       Freq   Term      Total  loglift  logprob
4  Default  60.000000    say  60.000000   8.0000   8.0000
0  Default  23.000000   know  23.000000   7.0000   7.0000
7  Default  41.000000    win  41.000000   6.0000   6.0000
2  Default  29.000000   love  29.000000   5.0000   5.0000
1  Default  35.000000  leave  35.000000   4.0000   4.0000
5  Default  17.000000  sleep  17.000000   3.0000   3.0000
6  Default  20.000000   tell  20.000000   2.0000   2.0000
3  Default  14.000000    man  14.000000   1.0000   1.0000
0   Topic1  23.006887   know  23.548471   0.5993  -1.7306
5   Topic1  17.371081  sleep  17.904259   0.5923  -2.0116
2   Topic1  27.253059   love  29.283024   0.5507  -1.5612
1   Topic1  31.450416  leave  35.020751   0.5150  -1.4180
6   Topic1  13.131647   tell  20.183402   0.1927  -2.2914
3   Topic1   8.372075    man  14.482225   0.0745  -2.7415
7   Topic1   8.423453    win  41.526588  -0.9728  -2.7354
4   Topic1   0.841793    say  60.051280  -3.6449  -5.0386
4   Topic2  59.209487    say  60.051280   0.7550  -0.6388
7   Topic2  33.103134    win  41.526588   0.5424  -1.2202
3   Topic2   6.110150    man  14.482225  -0.0939  -2.9099
6   Topic2   7.051755   tell  20.183402  -0.2825  -2.7666
1   Topic2   3.570335  leave  35.020751  -1.5142  -3.4472
2   Topic2   2.029965   love  29.283024  -1.8999  -4.0118
5   Topic2   0.533178  sleep  17.904259  -2.7448  -5.3487
0   Topic2   0.541584   know  23.548471  -3.0032  -5.3331, token_table=      Topic      Freq   Term
term                        
0         1  0.976709   know
0         2  0.042466   know
1         1  0.885189  leave
1         2  0.114218  leave
2         1  0.922036   love
2         2  0.068299   love
3         1  0.552401    man
3         2  0.414301    man
4         1  0.016652    say
4         2  0.982494    say
5         1  0.949495  sleep
5         2  0.055853  sleep
6         1  0.644094   tell
6         2  0.346820   tell
7         1  0.192648    win
7         2  0.794672    win, R=8, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [168]:
pyLDAvis.save_html(panel,'/Users/schlinkertc/code/NLP_practice/topics.htm')

In [169]:
df_document_topic['Song'] = hrefs

In [170]:
song_topics = df_document_topic.reset_index()

In [171]:
song_topics['Topic'] = song_topics['dominant_topic'].map({0:1,1:2})

In [172]:
song_topics.drop(columns={'Topic0','Topic1','index','dominant_topic'},inplace=True)

In [179]:
song_topics.set_index(['Topic','Song']).sort_index()

Empty DataFrame
Columns: []
Index: [(1, Be-My-Own-Bonus-Track), (1, Chastain), (1, Cut-The-Rope), (1, Deirdre-Pt-I), (1, Don-t-Let-Me), (1, Don-t-Say-Nothing), (1, High-Tops), (1, Homeless), (1, In-The-Yard), (1, Laid-Down-My-Arms), (1, Lamplight), (1, Lost-My-Cat-Put-in-a-Cage), (1, Love-Song-For-Lady-Earth-Sasha), (1, Still-in-Love), (1, Theory-of-Emotion), (1, To-Philly), (1, Vanessa), (2, Don-t-Read-The-Mirror), (2, I-Am-Drunk-And-She-Is-Insane), (2, Let-s-Pretend), (2, Rockman-s-Pier)]